# Lecture 05: Advanced SQL

In [1]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

If you haven't yet, load in the `imdb_lecture` and `stops_lecture` databases.

In [2]:
!mkdir data
!cp -v -r ../lec02/data .
!cp -v -r ../lec04/data/stops_lecture.sql data/.

'../lec02/data/imdb_lecture.zip' -> './data/imdb_lecture.zip'
'../lec02/data/imdb_lecture.sql' -> './data/imdb_lecture.sql'
'../lec02/data/.ipynb_checkpoints' -> './data/.ipynb_checkpoints'
'../lec04/data/stops_lecture.sql' -> 'data/./stops_lecture.sql'


In [3]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS imdb_lecture'
!psql -h localhost -c 'CREATE DATABASE imdb_lecture' 
!psql -h localhost -d imdb_lecture -f data/imdb_lecture.sql

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
SET
SET
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
COPY 8405
COPY 4043
COPY 4923
COPY 1223
COPY 820
COPY 2420


In [4]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS stops_lecture'
!psql -h localhost -c 'CREATE DATABASE stops_lecture' 
!psql -h localhost -d imdb_lecture -f data/stops_lecture.sql

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
SET
SET
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
COPY 84779
COPY 211
CREATE INDEX
CREATE INDEX


# Exercise #1: String manipulation

In [ ]:
%load_ext sql

In [ ]:
%%sql
SELECT 'Hello' || 'World',
       STRPOS('Hello', 'el'),
       SUBSTRING('Hello', 2, 3);

In [ ]:
%%sql
SELECT REGEXP_REPLACE('Hannah Montana', '(.*)', '\1') as ex1,
       REGEXP_REPLACE('Hannah Montana', '(.*) (.*)', '\2, \1') as ex2,
       REGEXP_REPLACE('Phone Number 510 642 3214', '[a-zA-Z ]', '') as ex3,
       REGEXP_REPLACE('Phone Number 510 642 3214', '[a-zA-Z ]', '', 'g') as ex4
;

# Exercise 2

In [ ]:
%load_ext sql

## Choice A

In [ ]:
%%sql
SELECT REGEXP_REPLACE(name, '(.*) (.*)', '\2') as lastname,
       REGEXP_REPLACE(name, '(.*) (.*)', '\1') as firstname
FROM people;

## Choice B

In [ ]:
%%sql
SELECT REGEXP_REPLACE(name, '(.*) (.*)', '\1') as firstname,
       COUNT(*) as countname
FROM people
GROUP BY firstname
ORDER BY countname desc;

## Choice C

In [ ]:
%%sql
WITH firstnames AS (
    SELECT REGEXP_REPLACE(name, '(.*) (.*)', '\1') as firstname
    FROM people )
SELECT MAX(LENGTH(firstname))
FROM firstnames;

What is this first name? Why doesn't the below query work?

In [ ]:
%%sql
WITH firstnames AS (
    SELECT REGEXP_REPLACE(name, '(.*) (.*)', '\1') as firstname
    FROM people )
SELECT MAX(LENGTH(firstname)), firstname
FROM firstnames;

Solution:

In [ ]:
%%sql
WITH firstnames AS (
    SELECT REGEXP_REPLACE(name, '(.*) (.*)', '\1') as firstname
    FROM people )
SELECT MAX(LENGTH(firstname)), firstname
FROM firstnames
GROUP BY firstname
ORDER BY max DESC;

# Exercise \#3: Grouping

In [ ]:
%reload_ext sql

In [ ]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS stops_lecture'
!psql -h localhost -c 'CREATE DATABASE stops_lecture' 
!psql -h localhost -d stops_lecture -f data/stops_lecture.sql

In [ ]:
%sql postgresql://127.0.0.1:5432/stops_lecture

In [ ]:
%%sql
SELECT COUNT(*)
FROM stops;

In [ ]:
%%sql
SELECT DISTINCT COUNT(*)
FROM Zips;

In [ ]:
%%sql
SELECT COUNT(*)
FROM Stops
    INNER JOIN Zips
    ON Stops.location = Zips.location;

# Demo: Windows and Casting

In [ ]:
%reload_ext sql

In [ ]:
%sql --close postgresql://127.0.0.1:5432/stops_lecture

In [ ]:
%sql postgresql://127.0.0.1:5432/imdb_lecture

Make a temporary other table

In [ ]:
%%sql
DROP TABLE IF EXISTS actor_title;
CREATE TABLE actor_title AS (
    SELECT t1.primary_title AS title, t1.title_id, a1.name
    FROM titles t1, people a1, crew c1
    WHERE t1.title_id = c1.title_id AND a1.person_id = c1.person_id
);


In [ ]:
%%sql
SELECT *
FROM actor_title
TABLESAMPLE BERNOULLI (5);

Choice A

In [ ]:
%%sql
SELECT title_id, name, title,
       AVG(LENGTH(name)) OVER (PARTITION BY title)
         AS avg_name_length
FROM actor_title
WHERE title LIKE 'The %'
ORDER BY title;

Choice B

In [ ]:
%%sql
SELECT title_id, name, title,
       CAST(AVG(LENGTH(name)) OVER (PARTITION BY title)
            AS INTEGER)
         AS avg_name_length
FROM actor_title
WHERE title LIKE 'The %'
ORDER BY title;

Choice C

In [ ]:
%%sql
SELECT title_id, name, title,
       LENGTH(name),
       RANK() OVER (PARTITION BY title ORDER BY LENGTH(name) DESC)
         AS name_rank
FROM actor_title
WHERE title LIKE 'A %'
ORDER BY title, name_rank;